## Prerequisites
### Standard imports

In [1]:
from __future__ import print_function, unicode_literals # Just in case of Python 2
import os
import json

### External libraries

To facilitate easier development, use existing libraries

[Requests-OAuthlib](https://requests-oauthlib.readthedocs.io)  
`pip install requests-oauthlib`  
An easy to use library for building OAuth1 and OAuth2 clients

[Tabulate](https://pypi.python.org/pypi/tabulate/)  
`pip install tabulate`  
Pretty-print tabular data

[Jupyter](http://jupyter.org)  
`pip install jupyter`  
This notebook is based on Jupyter (formerly IPython). Just using display to directly output tables

In [2]:
from requests_oauthlib import OAuth2Session
from tabulate import tabulate
from IPython import display

### Fix Python 2

In [3]:
try:
    input = raw_input
except NameError:
    pass

## Standard configuration

In [4]:
client_id = 'contestant'
client_secret = 'secret'
hostroot = 'http://api.beyondhackathon.com/'
scope = ['read', 'write']

## Custom configuration
This is the URL of your web application, where the OAuth code will end up

In [5]:
redirect_uri = 'http://example.com/auth_callback'

## Fiddler configuration

If you are using [Fiddler](http://www.getfiddler.com/), the following lines are required to make OAuth2 work with the proxy.

In [6]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1' # Ignore any HTTPS errors (for example the ones generated by Fiddler)
os.environ['http_proxy'] = 'http://localhost:8888' # default Fiddler port
os.environ['https_proxy'] = 'http://localhost:8888'

## Get authorization URL

In [7]:
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=scope)
authorization_url, state = oauth.authorization_url(hostroot + 'authorization/authorize')

## Visit the URL manually
You need to visit the URL from a browser and login as the end-user

In a real web application, you can simply redirect (302) the user

In [8]:
print('Please go to %s and authorize access.' % authorization_url)

Please go to http://api.beyondhackathon.com/authorization/authorize?response_type=code&client_id=contestant&redirect_uri=http%3A%2F%2Fexample.com%2Fauth_callback&scope=read+write&state=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX and authorize access.


## Manually enter the result URL
Since this is just a notebook and not a real web site, pretend you opened the authorize callback page. Just paste the URL that your browser has displayed.

In [9]:
authorization_response = input('Enter the full callback URL:')

Enter the full callback URL:http://example.com/auth_callback?code=XXXXXX&state=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


## User token for authorization

In [10]:
oauth_url = hostroot + 'authorization/token'
token = oauth.fetch_token(oauth_url, client_secret=client_secret, authorization_response=authorization_response)

## You are ready to use the API

## Retrieve customer info

In [11]:
customer_info_request = oauth.get(hostroot + 'customers/me/info')
me = customer_info_request.json()

In [12]:
print(me)

{'last_name': 'Frami', 'first_name': 'Aryana', 'mobile_numbers': ['+306919045271'], 'corporate': False, 'email_addresses': ['aframi@example.com']}


## Retrieve customer unread messages

In [13]:
message_count_request = oauth.get(hostroot + '/customers/me/messages-count')
message_count = message_count_request.json()['number']

In [14]:
print(message_count)

1


## Get list of product (i.e. cards and accounts)

In [15]:
products = oauth.get(hostroot + '/customers/me/products').json()

## Generate list of statements

In [16]:
statements = []
for p in products:
    if p['type'] == 'account':
        r3 = oauth.get(hostroot + '/accounts/{}/statement'.format(p['contract_number']))
    elif p['type'] == 'card':
        r3 = oauth.get(hostroot + '/cards/{}/statement'.format(p['contract_number']))
    for sl in r3.json():
        sl['type'] = p['type']
        sl['alias'] = p['alias']
        statements.append(sl)

## List statements as a table

Since they are dictionaries, the order is not guaranteed

In [17]:
display.Markdown(tabulate(statements, tablefmt='pipe', headers="keys", floatfmt=".2f"))

| alias     | type    |   new_balance |   amount | description                                  | transaction_date   | value_date   |
|:----------|:--------|--------------:|---------:|:---------------------------------------------|:-------------------|:-------------|
| Bisque    | account |       9727.69 |    20.69 | Κατάθεση 648                                 | 2016-01-06         | 2016-01-06   |
| Bisque    | account |       9651.88 |   -75.81 | Ανάληψη 238                                  | 2016-01-06         | 2016-01-06   |
| Bisque    | account |       9575.51 |   -76.37 | Πληρωμή Βεβαιωμένης Οφειλής 740              | 2016-01-07         | 2016-01-07   |
| Bisque    | account |       9502.35 |   -73.16 | Πληρωμή Βεβαιωμένης Οφειλής 002              | 2016-01-07         | 2016-01-07   |
| Bisque    | account |       9405.62 |   -96.73 | Ανάληψη 829                                  | 2016-01-07         | 2016-01-07   |
| Bisque    | account |       9418.63 |    13.01 | Κατάθεση 146                                 | 2016-01-07         | 2016-01-07   |
| Bisque    | account |       9378.96 |   -39.67 | Πληρωμή Βεβαιωμένης Οφειλής 780              | 2016-01-07         | 2016-01-07   |
| DimGray   | account |       8703.12 |   -88.88 | Πληρωμή Βεβαιωμένης Οφειλής 029              | 2016-04-22         | 2016-04-22   |
| DimGray   | account |       8793.47 |    90.35 | Κατάθεση 356                                 | 2016-04-22         | 2016-04-22   |
| Plum      | account |       4691.62 |     1.62 | Κατάθεση 448                                 | 2016-01-11         | 2016-01-11   |
| Plum      | account |       4749.53 |    57.91 | Κατάθεση 859                                 | 2016-01-11         | 2016-01-11   |
| LightGray | card    |        173.25 |   173.25 | Αγορά Distributed optimal approach           | 2016-01-25         | 2016-01-25   |
| LightGray | card    |        186.77 |    13.52 | Αγορά Stand-alone bi-directional time-frame  | 2016-01-26         | 2016-01-26   |
| LightGray | card    |        353.77 |   167.00 | Αγορά Reactive 24/7 alliance                 | 2016-01-26         | 2016-01-26   |
| DeepPink  | card    |        114.74 |   114.74 | Αγορά Future-proofed scalable implementation | 2016-01-18         | 2016-01-18   |